# Stationary Single-Group Pedestrian Flow with SUPG Stabilization

## 1. Introduction

This notebook implements a stationary (steady-state) single-group pedestrian flow model based on the Hughes model with **SUPG (Streamline Upwind Petrov-Galerkin) stabilization**. The model couples a continuity equation for pedestrian density with a Helmholtz equation for path planning.

**Key difference from standard formulation**: SUPG stabilization allows using **smaller diffusion** (ε = 0.01 m²) by adding streamline stabilization to handle convection-dominated flow, avoiding spurious oscillations that would occur with standard Galerkin method.

---

## 2. Strong Form Formulation

### 2.1 Domain and Variables

- **Domain:** $\Omega \subset \mathbb{R}^2$ (the region where pedestrians move)
- **Boundary:** $\partial\Omega = \Gamma_{\text{walls}} \cup \Gamma_{\text{exits}} \cup \Gamma_{\text{entries}}$

**Primary Variables:**
- $\rho(x,y)$ : pedestrian density [ped/m²]
- $\psi(x,y)$ : transformed potential (via Cole-Hopf transformation: $\psi = e^{-\Phi/\delta}$)
- $u(x,y)$ : velocity field [m/s]

### 2.2 Governing Equations

**Continuity Equation (with diffusion):**
$$
\nabla \cdot (-\varepsilon\nabla\rho + \rho u) = 0 \quad \text{in } \Omega
$$

**Helmholtz Equation (for path planning):**
$$
\Delta\psi - \frac{1}{\delta^2 f^2(\rho)} \psi = 0 \quad \text{in } \Omega
$$

**Velocity Field:**
$$
u = f(\rho) \frac{\nabla\psi}{\|\nabla\psi\|}
$$

where:
- $\varepsilon = 0.01$ m² = **reduced diffusion** (10x smaller than standard formulation)
- $\delta$ = viscosity parameter (regularization parameter)
- $f(\rho)$ = fundamental diagram (walking speed as function of density)

### 2.3 SUPG Stabilization

For convection-dominated problems (small ε), the standard Galerkin method produces spurious oscillations. SUPG adds **streamline diffusion** in the flow direction:

**SUPG-stabilized weak form:**
$$
\int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega + \sum_K \int_K \tau_K (u \cdot \nabla w)(\nabla \cdot (-\varepsilon\nabla\rho + \rho u)) \, d\Omega
$$

**Stabilization parameter:**
$$
\tau_K = \frac{h_K}{2\|u\|_K} \left(\text{coth}(\text{Pe}_K) - \frac{1}{\text{Pe}_K}\right)
$$

where:
- $h_K$ = element size
- $\text{Pe}_K = \frac{\|u\|_K h_K}{2\varepsilon}$ = local Péclet number
- Simplified: $\tau_K \approx \frac{h_K}{2\|u\|_K}$ for high Péclet number

### 2.4 Fundamental Diagram

We use Weidmann's fundamental diagram:
$$
f(\rho) = \begin{cases}
u_0 & \text{if } \rho = 0 \\
u_0\left(1 - e^{-\gamma(1/\rho - 1/\rho_c)}\right) & \text{if } 0 < \rho \leq \rho_c
\end{cases}
$$

**Parameters:**
- $u_0 = 1.36$ m/s (free-flow walking speed)
- $\rho_c = 8$ ped/m² (critical density)
- $\gamma = 1.913$ ped/m² (shape parameter)

### 2.5 Boundary Conditions

**On Walls ($\Gamma_{\text{walls}}$):**
- Density: $(-\varepsilon\nabla\rho + \rho u)\cdot n = 0$ (no flux through walls)
- Potential: $\nabla\psi\cdot n = 0$ (slip condition)

**On Exits ($\Gamma_{\text{exits}}$):**
- Density: $(-\varepsilon\nabla\rho)\cdot n = 0$ (free outflow)
- Potential: $\psi = 1$ (Dirichlet BC, minimal travel time at exit)

**On Entrances ($\Gamma_{\text{entries}}$):**
- Density: $-(-\varepsilon\nabla\rho + \rho u)\cdot n = g$ (prescribed influx)
- Potential: $(u_0\delta\nabla\psi)\cdot n + \psi = 0$ (Robin BC)

where:
- $n$ = outward normal vector on boundary
- $g$ = inflowing flux density [ped/(m·s)]

---

## 3. SUPG-Stabilized Weak Form for Continuity Equation

**Standard Galerkin (unstable for small ε):**
$$
\int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega = RHS
$$

**SUPG-Stabilized (stable for small ε):**
$$
\int_\Omega \varepsilon\nabla\rho \cdot \nabla w \, d\Omega - \int_\Omega (\rho u) \cdot \nabla w \, d\Omega + \sum_K \int_K \tau_K (u \cdot \nabla w)(u \cdot \nabla\rho) \, dK = RHS + RHS_{SUPG}
$$

The SUPG term adds **artificial diffusion only in the streamline direction**, preserving accuracy in cross-stream direction.

In [1]:
# ========================================
# Module Import
# ========================================

from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np

In [2]:
# ========================================
# Geometry and BC naming
# ========================================

mesh_maxh = 0.05
Hcol = 1.0  # Height [m]
Hwid = 1.0  # Width [m]

# Top rectangle
rect = Rectangle(Hwid, Hcol).Face()
rect.edges.Min(X).name = "left"
rect.edges.Max(X).name = "right"
rect.edges.Max(Y).name = "entry"
rect.edges.Min(Y).name = "exit"

geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")

Mesh: 514 vertices, 946 elements


## Model Parameters and Constants

We now define all physical parameters and numerical constants for the pedestrian flow model.

In [ ]:
# ========================================
# Physical Parameters (from Weidmann)
# ========================================
u0 = 1.36          # Free-flow walking speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

# ========================================
# Regularization Parameters
# ========================================
delta = 0.1        # Viscosity parameter [m]
epsilon = 0.01     # Diffusion coefficient [m²] - REDUCED for SUPG!
eta = 1e-8         # Gradient regularization (avoid division by zero)

# ========================================
# SUPG Stabilization Parameters
# ========================================
C_supg = 10       # SUPG stabilization constant (increased for ε=0.01)

# ========================================
# Picard Relaxation
# ========================================
omega = 0.05        # Underrelaxation parameter (REDUCED for ε=0.01)

# ========================================
# Boundary Conditions
# ========================================
g_inflow = 1.0     # Influx at entrance [ped/(m·s)]

# ========================================
# Numerical Parameters
# ========================================
p_order = 3        # Polynomial order of FEM spaces
max_iter = 500     # Maximum number of Picard iterations (increased for slow convergence)
tol = 1e-6         # Convergence tolerance for ||rho^(k+1) - rho^(k)||

print("Model Parameters:")
print(f"  u0 (free-flow speed)     = {u0} m/s")
print(f"  rho_c (critical density) = {rho_c} ped/m²")
print(f"  gamma (Weidmann param)   = {gamma_w} ped/m²")
print(f"\nRegularization:")
print(f"  delta (viscosity)        = {delta} m")
print(f"  epsilon (diffusion)      = {epsilon} m² *** REDUCED (10x) ***")
print(f"  eta (grad regularization)= {eta}")
print(f"\nSUPG Stabilization:")
print(f"  C_supg (stabilization)   = {C_supg}")
print(f"\nPicard Relaxation:")
print(f"  omega (underrelaxation)  = {omega} *** STRONG underrelaxation ***")
print(f"\nBoundary Conditions:")
print(f"  g_inflow (entrance flux) = {g_inflow} ped/(m·s)")
print(f"\nNumerical Settings:")
print(f"  max_iter                 = {max_iter}")
print(f"  tolerance                = {tol}")

Model Parameters:
  u0 (free-flow speed)     = 1.36 m/s
  rho_c (critical density) = 8.0 ped/m²
  gamma (Weidmann param)   = 1.913 ped/m²

Regularization:
  delta (viscosity)        = 0.1 m
  epsilon (diffusion)      = 0.01 m² *** REDUCED (10x) ***
  eta (grad regularization)= 1e-08

SUPG Stabilization:
  C_supg (stabilization)   = 10

Picard Relaxation:
  omega (underrelaxation)  = 0.05 *** STRONG underrelaxation ***

Boundary Conditions:
  g_inflow (entrance flux) = 1.0 ped/(m·s)

Numerical Settings:
  max_iter                 = 500
  tolerance                = 1e-06


In [4]:
# ============================================================================
# PARAMETER ANALYSIS - Verify numerical stability for SUPG simulation
# ============================================================================
from src import analyze_parameters

print("Running parameter analysis for SUPG-stabilized simulation...\n")
print("WARNING: With ε=0.01, expect HIGH Péclet numbers!\n")

results = analyze_parameters(
    u0=u0,
    rho_c=rho_c,
    gamma_w=gamma_w,
    delta=delta,
    epsilon=epsilon,
    mesh=mesh,
    mesh_maxh=mesh_maxh,
    p_order=p_order,
    Hwid=Hwid,
    Hcol=Hcol,
    omega=omega  # Include underrelaxation parameter
)

# For SUPG, we expect unstable Péclet numbers (that's why we use SUPG!)
if results['Pe_h'] > 1.0:
    print("\n" + "="*60)
    print("⚠ High Péclet number detected (Pe_h > 1)")
    print("  This is EXPECTED for SUPG stabilization!")
    print(f"  SUPG constant C_supg = {C_supg}")
    print(f"  Underrelaxation ω = {omega}")
    print("  Convergence may be slow but should stabilize.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("✓ Péclet number is low - SUPG may not be necessary!")
    print("  Consider using standard Galerkin with larger epsilon.")
    print("="*60)

Running parameter analysis for SUPG-stabilized simulation...




NameError: name 'p_order' is not defined

In [ ]:
# ========================================
# Weidmann Fundamental Diagram
# ========================================

def weidmann_speed(rho_val):
    """
    Weidmann fundamental diagram: f(rho)
    
    Returns walking speed as a function of density.
    
    f(rho) = u0 * (1 - exp(-gamma * (1/rho - 1/rho_c)))  for 0 < rho <= rho_c
    f(0)   = u0                                           for rho = 0
    
    Parameters:
    -----------
    rho_val : float or CoefficientFunction
        Pedestrian density [ped/m²]
    
    Returns:
    --------
    speed : float or CoefficientFunction
        Walking speed [m/s]
    """
    # Add small regularization to avoid division by zero
    rho_reg = IfPos(rho_val - 1e-10, rho_val, 1e-10)
    
    # Weidmann formula
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    # Ensure speed doesn't exceed u0 (free-flow speed)
    speed = IfPos(u0 - speed, speed, u0)
    
    # Ensure speed is non-negative
    speed = IfPos(speed, speed, 0.0)
    
    return speed


# Test function for pure Python evaluation (for testing)
def weidmann_speed_python(rho_val):
    """Python version for numerical testing"""
    rho_reg = max(rho_val, 1e-10)
    speed = u0 * (1 - np.exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    speed = min(max(speed, 0.0), u0)
    return speed


# Test the function with some sample values
print("Testing Weidmann fundamental diagram:")
print(f"  f(0.1) = {weidmann_speed_python(0.1):.4f} m/s")
print(f"  f(1.0) = {weidmann_speed_python(1.0):.4f} m/s")
print(f"  f(2.0) = {weidmann_speed_python(2.0):.4f} m/s")
print(f"  f(4.0) = {weidmann_speed_python(4.0):.4f} m/s")
print(f"  f(8.0) = {weidmann_speed_python(rho_c):.4f} m/s  (at critical density)")
print(f"  f(0.0) ≈ {weidmann_speed_python(1e-12):.4f} m/s  (near zero)")

In [ ]:
# ========================================
# Finite Element Spaces
# ========================================

# H1 space for density (rho) - no Dirichlet BCs
# Order 2 for better accuracy
fes_rho = H1(mesh, order=p_order)

# H1 space for potential (psi) - Dirichlet BC on exit
# We set dirichlet flag on "exit" boundary
fes_psi = H1(mesh, order=p_order, dirichlet="right")

# Vector H1 space for velocity field (u)
# Dimension 2 for 2D problem
fes_u = H1(mesh, order=p_order, dim=2)

print("Finite Element Spaces:")
print(f"  Density (rho):    {fes_rho.ndof} DOFs")
print(f"  Potential (psi):  {fes_psi.ndof} DOFs ({fes_psi.ndof - fes_rho.ndof} fixed by Dirichlet BC)")
print(f"  Velocity (u):     {fes_u.ndof} DOFs")

# ========================================
# Grid Functions (Solution Variables)
# ========================================

# Current solution
gf_rho = GridFunction(fes_rho, name="density")
gf_psi = GridFunction(fes_psi, name="potential")
gf_u = GridFunction(fes_u, name="velocity")

# Previous iteration (for Picard)
gf_rho_old = GridFunction(fes_rho, name="density_old")

# ========================================
# Initialize with reasonable values
# ========================================

# Initialize density with a small positive value
gf_rho.Set(0.1)
gf_rho_old.Set(0.1)

# Initialize potential with linear interpolation from entry (0) to exit (1)
# Exit is at y=0, entry at y=Hcol
gf_psi.Set(y / Hcol)  

# Set Dirichlet BC for psi: psi = 1 at exit (y=0)
gf_psi.Set(1.0, definedon=mesh.Boundaries("exit"))

# Velocity will be computed from psi
gf_u.Set((0, 0))

print("\nGrid Functions initialized:")
print(f"  rho: min = {min(gf_rho.vec):.4f}, max = {max(gf_rho.vec):.4f}")
print(f"  psi: min = {min(gf_psi.vec):.4f}, max = {max(gf_psi.vec):.4f}")

In [ ]:
# ========================================
# Helmholtz Equation Weak Form (for ψ)
# ========================================
# Same as standard formulation (no stabilization needed)

print("Setting up Helmholtz equation weak form...")

# Trial and test functions
psi = fes_psi.TrialFunction()
phi = fes_psi.TestFunction()

# Coefficient function for κ²(ρ) based on current density
f_rho = weidmann_speed(gf_rho)
kappa_sq = 1.0 / (delta**2 * f_rho**2)

# Bilinear form for Helmholtz equation
a_psi = BilinearForm(fes_psi, symmetric=True)
a_psi += grad(psi) * grad(phi) * dx
a_psi += kappa_sq * psi * phi * dx
a_psi += (1.0 / (u0 * delta)) * psi * phi * ds("entry")

# Linear form (RHS)
L_psi = LinearForm(fes_psi)

print("  ✓ Helmholtz weak form created (standard formulation)")

# ========================================
# Continuity Equation with SUPG Stabilization
# ========================================

print("\nSetting up SUPG-stabilized Continuity equation...")

# Trial and test functions
rho = fes_rho.TrialFunction()
w = fes_rho.TestFunction()

# Mesh size (elementwise)
h = specialcf.mesh_size

# Bilinear form for Continuity equation
a_rho = BilinearForm(fes_rho, symmetric=False)

# Standard Galerkin terms
# Diffusion: ∫_Ω ε∇ρ·∇w dΩ
a_rho += epsilon * grad(rho) * grad(w) * dx

# Convection: -∫_Ω (ρu)·∇w dΩ
a_rho += -rho * (gf_u * grad(w)) * dx

# Exit boundary: ∫_Γ_exit (ρu·n)w dS
n = specialcf.normal(2)
a_rho += rho * (gf_u * n) * w * ds("right")

# SUPG Stabilization term
# τ = C_supg * h / (2||u||)
# SUPG: ∫_K τ (u·∇w)(u·∇ρ) dK
u_norm = sqrt(gf_u[0]**2 + gf_u[1]**2 + 1e-10)  # Avoid division by zero
tau_supg = C_supg * h / (2 * u_norm)

# SUPG term: streamline diffusion
a_rho += tau_supg * (gf_u * grad(w)) * (gf_u * grad(rho)) * dx

# Linear form (RHS)
L_rho = LinearForm(fes_rho)
L_rho += g_inflow * w * ds("entry")

print("  ✓ SUPG-stabilized Continuity weak form created")
print(f"    - Standard Galerkin terms:")
print(f"      • Diffusion: ∫ ε∇ρ·∇w dx  (ε = {epsilon})")
print(f"      • Convection: -∫ (ρu)·∇w dx")
print(f"      • Exit BC: ∫ (ρu·n)w ds")
print(f"    - SUPG stabilization:")
print(f"      • τ = {C_supg} * h / (2||u||)")
print(f"      • SUPG term: ∫ τ (u·∇w)(u·∇ρ) dx")

print("\n✓ Weak forms setup complete")

In [ ]:
# ========================================
# Velocity Field Computation
# ========================================
# Computes: u = f(ρ) * ∇ψ / (||∇ψ|| + η)
# where f(ρ) is the Weidmann speed and η is a regularization parameter

def compute_velocity():
    """
    Compute velocity field from current potential and density.
    
    Updates gf_u based on:
        u = f(rho) * grad(psi) / (||grad(psi)|| + eta)
    
    This is the coupling between Helmholtz and Continuity equations.
    """
    # Compute gradient of potential
    grad_psi = grad(gf_psi)
    
    # Compute norm of gradient with regularization
    grad_psi_norm = sqrt(grad_psi[0]**2 + grad_psi[1]**2 + eta)
    
    # Compute speed from current density
    speed = weidmann_speed(gf_rho)
    
    # Compute velocity: u = f(rho) * grad(psi) / ||grad(psi)||
    velocity_cf = speed * grad_psi / grad_psi_norm
    
    # Set the velocity grid function
    gf_u.Set(velocity_cf)
    
    return gf_u


print("Velocity field computation function defined:")
print("  u = f(ρ) * ∇ψ / (||∇ψ|| + η)")
print(f"  - f(ρ): Weidmann fundamental diagram")
print(f"  - η = {eta} (gradient regularization)")
print("\n✓ Velocity computation ready")

In [ ]:
# ========================================
# Picard Iteration Solver with SUPG and Underrelaxation
# ========================================

def solve_stationary_picard_supg():
    """
    Solve the coupled stationary pedestrian flow problem using Picard iteration
    with SUPG stabilization and underrelaxation.
    
    Algorithm:
    1. Initialize with current gf_rho
    2. Loop:
       a. Solve Helmholtz for psi (using current rho)
       b. Compute velocity u from psi
       c. Solve SUPG-stabilized Continuity for rho (using current u)
       d. Apply underrelaxation: ρ_new = ω*ρ_new + (1-ω)*ρ_old
       e. Check convergence
    """
    
    print("=" * 60)
    print("Starting Picard Iteration with SUPG & Underrelaxation")
    print("=" * 60)
    print(f"  ε = {epsilon} m² (reduced diffusion)")
    print(f"  SUPG constant = {C_supg}")
    print(f"  Underrelaxation ω = {omega}")
    print("=" * 60)
    
    residuals = []
    
    for iteration in range(max_iter):
        
        # Store old density
        gf_rho_old.vec.data = gf_rho.vec
        
        # ====================================
        # Step a: Solve Helmholtz equation
        # ====================================
        
        f_rho = weidmann_speed(gf_rho)
        kappa_sq = 1.0 / (delta**2 * f_rho**2)
        
        a_psi = BilinearForm(fes_psi, symmetric=True)
        a_psi += grad(psi) * grad(phi) * dx
        a_psi += kappa_sq * psi * phi * dx
        a_psi += (1.0 / (u0 * delta)) * psi * phi * ds("entry")
        
        a_psi.Assemble()
        L_psi.Assemble()
        
        gf_psi.Set(1.0, definedon=mesh.Boundaries("right"))
        gf_psi.vec.data += a_psi.mat.Inverse(fes_psi.FreeDofs()) * (L_psi.vec - a_psi.mat * gf_psi.vec)
        
        # ====================================
        # Step b: Compute velocity field
        # ====================================
        compute_velocity()
        
        # ====================================
        # Step c: Solve SUPG-stabilized Continuity
        # ====================================
        
        # Mesh size and SUPG parameter
        h = specialcf.mesh_size
        u_norm = sqrt(gf_u[0]**2 + gf_u[1]**2 + 1e-10)
        tau_supg = C_supg * h / (2 * u_norm)
        
        # Rebuild bilinear form with SUPG
        a_rho = BilinearForm(fes_rho, symmetric=False)
        
        # Standard Galerkin terms
        a_rho += epsilon * grad(rho) * grad(w) * dx
        a_rho += -rho * (gf_u * grad(w)) * dx
        n = specialcf.normal(2)
        a_rho += rho * (gf_u * n) * w * ds("right")
        
        # SUPG stabilization
        a_rho += tau_supg * (gf_u * grad(w)) * (gf_u * grad(rho)) * dx
        
        with TaskManager():
            a_rho.Assemble()
            L_rho.Assemble()
            
            # Solve for new density (temporarily store in gf_rho)
            rho_new = a_rho.mat.Inverse(fes_rho.FreeDofs()) * L_rho.vec
            
            # ====================================
            # Step d: Apply underrelaxation
            # ====================================
            # ρ^(k+1) = ω*ρ_new + (1-ω)*ρ^(k)
            gf_rho.vec.data = omega * rho_new + (1.0 - omega) * gf_rho_old.vec
        
        # ====================================
        # Step e: Check convergence
        # ====================================
        
        residual_vec = gf_rho.vec - gf_rho_old.vec
        residual = sqrt(InnerProduct(residual_vec, residual_vec))
        residuals.append(residual)
        
        # Print progress every 5 iterations or on convergence
        if iteration % 5 == 0 or residual < tol:
            rho_min = min(gf_rho.vec)
            rho_max = max(gf_rho.vec)
            print(f"Iter {iteration:3d}: residual = {residual:.6e}, "
                  f"rho ∈ [{rho_min:.4f}, {rho_max:.4f}]")
        
        # Check convergence
        if residual < tol:
            print("=" * 60)
            print(f"✓ Converged in {iteration + 1} iterations!")
            print(f"  Final residual: {residual:.6e}")
            print("=" * 60)
            return True, iteration + 1, residual, residuals
    
    # Max iterations reached
    print("=" * 60)
    print(f"✗ Did not converge in {max_iter} iterations")
    print(f"  Final residual: {residual:.6e}")
    print("=" * 60)
    return False, max_iter, residual, residuals


# Run the SUPG-stabilized solver with underrelaxation
converged, n_iter, final_residual, residuals = solve_stationary_picard_supg()

In [ ]:
# ========================================
# Visualize Density and Velocity
# ========================================

print("=" * 60)
print("Solution Visualization")
print("=" * 60)

# Compute domain area
domain_area = Integrate(1.0, mesh)

# Create order-1 space for min/max evaluation
# (Order-1 DOFs correspond to vertex values, making min/max meaningful)
fes_p1 = H1(mesh, order=1)

# Print solution statistics
print("\nSolution Statistics:")
print(f"  Density (ρ):")
rho_integral = Integrate(gf_rho * dx, mesh)
rho_mean = rho_integral / domain_area

# Interpolate to P1 for min/max
gf_rho_p1 = GridFunction(fes_p1)
gf_rho_p1.Set(gf_rho)

print(f"    integral = {rho_integral:.6f} ped")
print(f"    mean = {rho_mean:.6f} ped/m²")
print(f"    min ≈ {min(gf_rho_p1.vec):.6f} ped/m² (at vertices)")
print(f"    max ≈ {max(gf_rho_p1.vec):.6f} ped/m² (at vertices)")

print(f"\n  Domain area: {domain_area:.6f} m²")

print("=" * 60)

# Draw the fields
print("\nGenerating visualizations...")

# 1. Density field
Draw(gf_rho, mesh, "density")
print("  ✓ Density field (ρ)")

# 2. Velocity vector field
Draw(gf_u, mesh, "velocity", vectors={"grid_size": 30})
print("  ✓ Velocity vector field (u)")

print("\n✓ Visualization complete!")